In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

## Reading in the data

In [43]:
status = pd.read_csv("../data/financials/status_full.csv")

In [44]:
financials = pd.read_csv("../data/financials/financials_full.csv")

In [45]:
financials['default'] = [0]*financials.shape[0]

In [46]:
financials['date_diff'] = [np.nan]*financials.shape[0]

In [47]:
financials['default_date'] = [np.nan]*financials.shape[0]

In [48]:
financials['status'] = [np.nan]*financials.shape[0]

In [49]:
financials['actual_status'] = [np.nan]*financials.shape[0]

In [50]:
financials['remark'] = [np.nan]*financials.shape[0]

In [51]:
financials.shape

(481988, 42)

In [52]:
financials.head()

,company,id,year,net_sales,other_sales,op_profit_ebit,prof_after_net_fin_items,results,sub_unpaid_cap,fixed_assets,...,gross_prof,working_cap,solidity_equity_assets_ratio,cash_liquidity_quick_ratio,status_bool,date_diff,default_date,status,actual_status,remark
0,Lily Properties AB,5590655865,2019-12,0,-,-44,-44,-44,0,99,...,-,-,68.63%,57.14%,0,NaN,NaN,NaN,NaN,NaN
1,Lily Properties AB,5590655865,2018-12,0,-,-72,-72,-72,0,130,...,-,-,75.13%,81.63%,0,NaN,NaN,NaN,NaN,NaN
2,Lily Properties AB,5590655865,2017-12,28,-,-279,-279,-279,0,160,...,100.00%,1282.14%,91.23%,764.00%,0,NaN,NaN,NaN,NaN,NaN
3,Lily Properties AB,5590655865,2016-12,0,-,-226,749,749,0,801,...,-,-,94.22%,40.82%,0,NaN,NaN,NaN,NaN,NaN
4,Fogdetorps El Aktiebolag,5564709904,2019-12,0,-,-62,7938,7938,0,8649,...,-,-,74.29%,269.62%,0,NaN,NaN,NaN,NaN,NaN


## Removing the ids from status that do not appear in the financials dataset and visa versa

In [53]:
financials['id'].nunique()

69510

In [54]:
status['id'].nunique()

81335

In [55]:
status = status[status['id'].isin(financials['id'].unique())]

In [56]:
status.shape

(69331, 6)

In [57]:
status['id'].nunique()

69326

In [58]:
financials['id'].nunique()

69510

In [59]:
financials = financials[financials['id'].isin(status['id'].unique())]

In [60]:
financials['id'].nunique()

69326

In [61]:
status[status['status']=='inaktivt'].shape

(2139, 6)

## Number of companies that are inaktivt but not banckrupt

To get around this we can just set the boolean column to 1 for the most recent year showing financials

In [62]:
status[status['status']=='inaktivt']['id'].nunique()-status[status['status']=='inaktivt'][~status['remark'].isna()]['id'].nunique()

331

In [63]:
indexnames = status[status['status']=='inaktivt'][status['remark'].isna()]['id'].index

In [25]:
indexnames.shape

(332,)

In [26]:
# removing companies that are inactive but are not bankrupt (from status and financial)
status = status.drop(indexnames)
financials = financials[financials['id'].isin(status['id'].unique())]

In [27]:
financials['id'].nunique()

68995

In [28]:
status[status['status']=='inaktivt'].shape

(1807, 6)

In [29]:
status.shape

(68999, 6)

In [30]:
financials.shape

(478042, 42)

## Coming up with a way to merge the status and financials data

In [31]:
status[status['status']=='inaktivt'][500:505]

,company,id,status,actual_status,remark,default_date
20682,RmB Gruppen AB,5568159189,inaktivt,Registrerad,Konkurs inledd,2019-01-28
20733,Racon Bygg AB,5567767891,inaktivt,Registrerad,Konkurs inledd,2017-07-06
20734,B o B Riv o Sanering AB,5569808487,inaktivt,Registrerad,Konkurs inledd,2019-05-02
20763,Geo08 mätningsteknik AB,5568209497,inaktivt,Registrerad,Konkurs inledd,2020-01-23
20772,Bygg Klokt Stockholm AB,5569318628,inaktivt,Registrerad,Konkurs inledd,2020-11-25


In [32]:
company_test = financials[financials['id']==5569994840]

In [33]:
company_test

,company,id,year,net_sales,other_sales,op_profit_ebit,prof_after_net_fin_items,results,sub_unpaid_cap,fixed_assets,...,gross_prof,working_cap,solidity_equity_assets_ratio,cash_liquidity_quick_ratio,status_bool,date_diff,default_date,status,actual_status,remark
107914,Aktiebolaget husbyggarna i kungens kurva,5569994840,2017-12,22525,-,786,542,398,0,2941,...,37.50%,0.58%,28.40%,102.03%,0,NaN,NaN,NaN,NaN,NaN
107915,Aktiebolaget husbyggarna i kungens kurva,5569994840,2016-07,4866,-,212,209,163,0,2239,...,35.47%,1.11%,59.43%,103.50%,0,NaN,NaN,NaN,NaN,NaN
107916,Aktiebolaget husbyggarna i kungens kurva,5569994840,2015-07,0,-,0,0,0,0,810,...,-,-,99.38%,8000.00%,0,NaN,NaN,NaN,NaN,NaN


In [34]:
duration = datetime.strptime('2019-03-25', '%Y-%m-%d')-datetime.strptime(company_test['year'].iloc[1], '%Y-%m')
duration.days

997

In [35]:
(datetime.strptime(status[status['id']==5569994840]['default_date'].iloc[0],'%Y-%m-%d') - datetime.strptime(company_test['year'].iloc[0], '%Y-%m')).days

479

In [36]:
def add_company_status(company_id):
    company = financials[financials['id']==company_id]
    company_status = status[status['id']==company_id]
    years = company.shape[0]
    min_date_diff = 1000000
    min_date_diff_row = 0
    
    for i in range(years):
        date_diff = datetime.strptime(status[status['id']==company_id]['default_date'].iloc[0],'%Y-%m-%d') - datetime.strptime(company['year'].iloc[i], '%Y-%m')
        date_diff = date_diff.days
        if date_diff < 0:
            # returned financials after banckrupcy (should not happen)
            continue
        if date_diff < min_date_diff:
            min_date_diff = date_diff
            min_date_diff_row = i
    
    company['status_bool'].iloc[min_date_diff_row] = 1
    company['date_diff'].iloc[min_date_diff_row] = round(min_date_diff/365,3)
    company['default_date'].iloc[min_date_diff_row] = company_status['default_date'].iloc[0]
    company['status'].iloc[min_date_diff_row] = company_status['status'].iloc[0]
    company['actual_status'].iloc[min_date_diff_row] = company_status['actual_status'].iloc[0]
    company['remark'].iloc[min_date_diff_row] = company_status['remark'].iloc[0]
    
    return company

In [ ]:
count = 0
for Id in status['id'].unique():
    financials[financials['id']==Id] = add_company_status(Id)
    count+=1
count

In [38]:
status[status['id']==5567767891]

,company,id,status,actual_status,remark,default_date
20733,Racon Bygg AB,5567767891,inaktivt,Registrerad,Konkurs inledd,2017-07-06


In [38]:
financials[financials['id']==5567767891]

,company,id,year,net_sales,other_sales,op_profit_ebit,prof_after_net_fin_items,results,sub_unpaid_cap,fixed_assets,...,gross_prof,working_cap,solidity_equity_assets_ratio,cash_liquidity_quick_ratio,status_bool,date_diff,default_date,status,actual_status,remark
115760,Racon Bygg AB,5567767891,2016-12,13080,137,-10,-33,-33,0,169,...,33.37%,0.79%,8.68%,100.32%,1,0.595,2017-07-06,inaktivt,Registrerad,Konkurs inledd
115761,Racon Bygg AB,5567767891,2015-12,15723,96,77,56,41,0,351,...,31.19%,0.56%,9.83%,104.22%,0,NaN,NaN,NaN,NaN,NaN
115762,Racon Bygg AB,5567767891,2014-12,8263,174,513,493,434,0,437,...,47.31%,4.91%,19.66%,121.48%,0,NaN,NaN,NaN,NaN,NaN
115763,Racon Bygg AB,5567767891,2013-12,4661,18,-266,-283,-245,0,458,...,45.05%,-1.01%,6.56%,95.95%,0,NaN,NaN,NaN,NaN,NaN
115764,Racon Bygg AB,5567767891,2012-12,4460,63,-17,-19,-19,0,32,...,55.61%,3.97%,19.00%,121.61%,0,NaN,NaN,NaN,NaN,NaN
115765,Racon Bygg AB,5567767891,2011-12,4045,-,148,148,85,0,37,...,61.53%,4.62%,21.67%,124.44%,0,NaN,NaN,NaN,NaN,NaN
115766,Racon Bygg AB,5567767891,2010-12,5886,-,-64,-62,-62,0,86,...,56.39%,0.27%,8.61%,101.48%,0,NaN,NaN,NaN,NaN,NaN
115767,Racon Bygg AB,5567767891,2009-12,1763,7,5,5,3,0,61,...,66.53%,16.39%,16.38%,203.21%,0,NaN,NaN,NaN,NaN,NaN


In [39]:
status.head()

,company,id,status,actual_status,remark,default_date
2,Lily Properties AB,5590655865,aktivt,aktivt,NaN,NaN
4,Fogdetorps El Aktiebolag,5564709904,aktivt,Registrerad,NaN,NaN
12,Årsta Markentreprenad AB,5592051295,aktivt,aktivt,NaN,NaN
25,Sportgolvsgruppen Sverige AB,5568855877,aktivt,aktivt,NaN,NaN
26,J.N. Kraftinstallatörer Aktiebolag,5591365738,aktivt,aktivt,NaN,NaN


In [40]:
financials[financials['status_bool']==1]['id'].nunique()

1807

In [41]:
#financials.to_csv("../data/financials/financials_and_status.csv", index=False)

In [42]:
financials.shape

(478042, 42)

In [43]:
financials['id'].nunique()

68995

## Checking the merged dataset

In [53]:
fin_stat = pd.read_csv("../data/financials/financials_and_status.csv")

In [54]:
fin_stat.head()

,company,id,year,net_sales,other_sales,op_profit_ebit,prof_after_net_fin_items,results,sub_unpaid_cap,fixed_assets,...,gross_prof,working_cap,solidity_equity_assets_ratio,cash_liquidity_quick_ratio,status_bool,date_diff,default_date,status,actual_status,remark
0,Lily Properties AB,5590655865,2019-12,0,-,-44,-44,-44,0,99,...,-,-,68.63%,57.14%,0,NaN,NaN,NaN,NaN,NaN
1,Lily Properties AB,5590655865,2018-12,0,-,-72,-72,-72,0,130,...,-,-,75.13%,81.63%,0,NaN,NaN,NaN,NaN,NaN
2,Lily Properties AB,5590655865,2017-12,28,-,-279,-279,-279,0,160,...,100.00%,1282.14%,91.23%,764.00%,0,NaN,NaN,NaN,NaN,NaN
3,Lily Properties AB,5590655865,2016-12,0,-,-226,749,749,0,801,...,-,-,94.22%,40.82%,0,NaN,NaN,NaN,NaN,NaN
4,Fogdetorps El Aktiebolag,5564709904,2019-12,0,-,-62,7938,7938,0,8649,...,-,-,74.29%,269.62%,0,NaN,NaN,NaN,NaN,NaN


In [55]:
fin_stat.shape

(478042, 42)

In [56]:
fin_stat['id'].nunique()

68995

In [57]:
fin_stat[fin_stat['status_bool']==1].shape

(1807, 42)

In [60]:
fin_stat[fin_stat['status']=='inaktivt']

,company,id,year,net_sales,other_sales,op_profit_ebit,prof_after_net_fin_items,results,sub_unpaid_cap,fixed_assets,...,gross_prof,working_cap,solidity_equity_assets_ratio,cash_liquidity_quick_ratio,status_bool,date_diff,default_date,status,actual_status,remark
136,SH.EL.SA BYGG AB,5590846506,2017-12,0,-,0,0,0,0,0,...,-,-,100.00%,-,1,2.436,2020-05-08,inaktivt,Registrerad,Konkurs inledd
199,Ergo3 Bygg AB,5569664351,2019-06,0,-,-11,-11,-11,0,0,...,-,-,99.44%,17800.00%,1,1.244,2020-08-28,inaktivt,Registrerad,Konkurs inledd
645,Indust Bygg & Entreprenad AB,5590706205,2017-12,0,-,0,0,0,0,0,...,-,-,100.00%,-,1,2.049,2019-12-19,inaktivt,Registrerad,Konkurs inledd
671,SWEMMN KONSULT AB,5591180707,2018-06,0,-,-50,-50,-50,0,0,...,-,-,-,-,1,2.488,2020-11-25,inaktivt,Registrerad,Konkurs inledd
842,Bygglandia AB,5590842430,2018-06,0,-,-11,-11,-11,0,0,...,-,-,100.00%,-,1,2.507,2020-12-02,inaktivt,Registrerad,Konkurs inledd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477978,ÖBL Ventilation AB,5565872032,2017-12,17576,38,-1361,-1381,-501,0,14,...,35.23%,0.69%,5.01%,90.96%,1,2.647,2020-07-24,inaktivt,inaktivt,Konkurs avslutad
478006,Östmans Plåtslageri Stockholm AB,5569643850,2016-12,3106,6,-85,-108,-108,0,1219,...,73.50%,41.24%,7.27%,237.63%,1,2.849,2019-10-07,inaktivt,inaktivt,Konkurs avslutad
478009,Östra södermanland måleri bygg och konsulterin...,5568143696,2016-12,12960,230,80,-93,-303,0,91,...,75.82%,0.40%,4.51%,92.70%,1,2.548,2019-06-19,inaktivt,inaktivt,Konkurs avslutad
478015,Öxabäcks Måleri Aktiebolag,5561859694,2017-12,5557,8,-1795,-1876,-1586,0,53,...,57.33%,-18.05%,-209.62%,41.65%,1,1.951,2019-11-13,inaktivt,inaktivt,Konkurs avslutad


In [114]:
financials[financials['id']==5569643850]

,company,id,year,net_sales,other_sales,op_profit_ebit,prof_after_net_fin_items,results,sub_unpaid_cap,fixed_assets,...,gross_prof,working_cap,solidity_equity_assets_ratio,cash_liquidity_quick_ratio,status_bool,date_diff,default_date,status,actual_status,remark
481952,Östmans Plåtslageri Stockholm AB,5569643850,2016-12,3106,6,-85,-108,-108,0,1219,...,73.50%,41.24%,7.27%,237.63%,0,NaN,NaN,NaN,NaN,NaN
481953,Östmans Plåtslageri Stockholm AB,5569643850,2015-12,3832,-,76,69,69,0,0,...,55.69%,9.24%,42.63%,174.68%,0,NaN,NaN,NaN,NaN,NaN
481954,Östmans Plåtslageri Stockholm AB,5569643850,2014-12,1318,-,-317,-326,-326,0,0,...,64.49%,30.27%,12.43%,244.04%,0,NaN,NaN,NaN,NaN,NaN


In [3]:
board = pd.read_csv("../data/financials/board_test100.csv")

In [5]:
board[board['number_of_board_memebers'].isna()]

,company,id,number_of_board_memebers,average_age_of_board_members,total_active_assignments,total_prev_assignments
22,SH.EL.SA BYGG AB,5590846506,NaN,NaN,NaN,NaN
85,MGH Entreprenad Aktiebolag,5564269636,NaN,NaN,2.0,3.0
